In [1]:
import io
import paramiko

In [2]:
class KeyReader(object):
    
    available_pkey_classes = (
        paramiko.rsakey.RSAKey,
        paramiko.dsskey.DSSKey, 
        paramiko.ecdsakey.ECDSAKey,
        paramiko.ed25519key.Ed25519Key
    )
    
    def __init__(self, *args, **kwargs):
        self.__key = None
        
    def __load_certificate(self, cert_path, cert_data):
        if cert_path is not None:
            self.__key.load_certificate(cert_path)
        if cert_data is not None:
            self.__key.load_certificate(cert_data)

    def __read_private_key_from_file(self, filename, password):
        for pkey_class in self.available_pkey_classes:
            try:
                self.__key = pkey_class.from_private_key_file(filename, password)
            except paramiko.SSHException as e:
                print("{}".format(e))
            except paramiko.PasswordRequiredException:
                raise paramiko.PasswordRequiredException('Password is requred')

    def __read_private_key(self, pkey_str, password):
        for pkey_class in self.available_pkey_classes:
            try:
                file_obj = io.StringIO(pkey_str)
                self.__key = pkey_class.from_private_key(file_obj, password)
            except paramiko.SSHException as e:            
                print("{}".format(e))
            except paramiko.PasswordRequiredException:
                raise paramiko.PasswordRequiredException('Password is requred')
            finally:
                file_obj.close()
    
    def read(self, filename=None, pkey_str=None, password=None, cert_path=None, cert_data=None):
        if filename is not None:
            self.__read_private_key_from_file(filename, password)
        if pkey_str is not None:
            self.__read_private_key(pkey_str, password)
            
        if self.__key is None:
            raise paramiko.SSHException("Incorrect input credentials")
        else:
            self.__load_certificate(cert_path, cert_data)
            
        return self.__key


In [3]:
class SSHClient(object):
    def __init__(self, hostname, username, password=None, pkey=None):
        self.hostname = hostname
        self.username = username
        self.password = password
        self.pkey = pkey
        
        self.__client = paramiko.SSHClient()
        self.__client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
    
    def connect(self):
        self.__client.connect(hostname=self.hostname, username=self.username, password=self.password, pkey=self.pkey)
        
    def close(self):
        self.__client.close()
    
    def exec_command(self, cmd):
        return self.__client.exec_command(cmd)


In [4]:
HOST = '192.168.56.111'
USER = 'u1'
PASSWORD = 'q'
KEY_PASSPHRASE = 'P@ssw0rd'
KEY_FILE = '/home/shabykivai/.ssh/vm5/id_rsa'
KEY_CERTIFICATE_FILE = '/home/shabykivai/.ssh/vm5/id_rsa-cert.pub'

In [5]:
key_reader = KeyReader()
key = key_reader.read(filename=KEY_FILE, password=KEY_PASSPHRASE, cert_path=KEY_CERTIFICATE_FILE)
    
client = SSHClient(HOST, USER, PASSWORD, key)
client.connect()

not a valid DSA private key file
not a valid EC private key file
not a valid OPENSSH private key file


In [6]:
import abc

class SystemInfoCrawler(abc.ABC):
    @abc.abstractmethod
    def get_operating_system(self):
        raise NotImplementedError('Метод не реализован')
    
    @abc.abstractmethod
    def get_cpu(self):
        raise NotImplementedError('Метод не реализован') 
    
    @abc.abstractmethod
    def get_gpu(self):
        raise NotImplementedError('Метод не реализован') 
    
    @abc.abstractmethod
    def get_deamon_services(self):
        raise NotImplementedError('Метод не реализован')
    
    @abc.abstractmethod
    def get_software(self):
        raise NotImplementedError('Метод не реализован')


In [7]:
stdin, stdout, stderr = client.exec_command('cat /etc/os-release')
for line in stdout:
    print(line.strip('\n'))
    
stdin, stdout, stderr = client.exec_command('hostnamectl')
for line in stdout:
    print(line.strip('\n'))

NAME="Ubuntu"
VERSION="18.04.3 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.3 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
   Static hostname: u1
         Icon name: computer-vm
           Chassis: vm
        Machine ID: ca6dfc4eb469493fa214dcef38a759f5
           Boot ID: 1f0e0f02265545a7b98d01fcb71f4e60
    Virtualization: oracle
  Operating System: Ubuntu 18.04.3 LTS
            Kernel: Linux 5.0.0-23-generic
      Architecture: x86-64


In [8]:
import re 

def remove_quotes_from_line(line):
    return re.sub(r'[\"]', '', line)


def is_line_splitable(line, regex=r'[\=]'):
    return re.search(regex, line)


def split_line_to_tuple(line, by='='):
    items = line.split(by)
    return re.sub(' ', '_', items[0]).strip().lower(), items[-1].strip()


def parse_cat_os_release_stdout(stdout):
    return dict(split_line_to_tuple(remove_quotes_from_line(line.strip('\n')), '=') for line in stdout if is_line_splitable(line))


def parse_hostnamectl_stdout(stdout):
    return dict(split_line_to_tuple(remove_quotes_from_line(line.strip()), ':') for line in stdout if is_line_splitable(line, r'[\:]'))


In [9]:
stdin, stdout, stderr = client.exec_command('cat /etc/os-release')

parse_cat_os_release_stdout(stdout)

{'name': 'Ubuntu',
 'version': '18.04.3 LTS (Bionic Beaver)',
 'id': 'ubuntu',
 'id_like': 'debian',
 'pretty_name': 'Ubuntu 18.04.3 LTS',
 'version_id': '18.04',
 'home_url': 'https://www.ubuntu.com/',
 'support_url': 'https://help.ubuntu.com/',
 'bug_report_url': 'https://bugs.launchpad.net/ubuntu/',
 'privacy_policy_url': 'https://www.ubuntu.com/legal/terms-and-policies/privacy-policy',
 'version_codename': 'bionic',
 'ubuntu_codename': 'bionic'}

In [10]:
stdin, stdout, stderr = client.exec_command('hostnamectl')
    
parse_hostnamectl_stdout(stdout)

{'static_hostname': 'u1',
 'icon_name': 'computer-vm',
 'chassis': 'vm',
 'machine_id': 'ca6dfc4eb469493fa214dcef38a759f5',
 'boot_id': '1f0e0f02265545a7b98d01fcb71f4e60',
 'virtualization': 'oracle',
 'operating_system': 'Ubuntu 18.04.3 LTS',
 'kernel': 'Linux 5.0.0-23-generic',
 'architecture': 'x86-64'}

In [11]:
stdin, stdout, stderr = client.exec_command('lscpu')
for line in stdout:
    print(line.strip('\n'))

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              1
On-line CPU(s) list: 0
Thread(s) per core:  1
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               158
Model name:          Intel(R) Core(TM) i7-8700K CPU @ 3.70GHz
Stepping:            10
CPU MHz:             3695.976
BogoMIPS:            7391.95
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            12288K
NUMA node0 CPU(s):   0
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 syscall nx rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq monitor ssse3 cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti

In [12]:
def parse_lscpu_stdout(stdout):
    return [dict(split_line_to_tuple(remove_quotes_from_line(line), ':') for line in stdout if is_line_splitable(line, r'[\:]'))]


In [13]:
stdin, stdout, stderr = client.exec_command('lscpu')

parse_lscpu_stdout(stdout)

[{'architecture': 'x86_64',
  'cpu_op-mode(s)': '32-bit, 64-bit',
  'byte_order': 'Little Endian',
  'cpu(s)': '1',
  'on-line_cpu(s)_list': '0',
  'thread(s)_per_core': '1',
  'core(s)_per_socket': '1',
  'socket(s)': '1',
  'numa_node(s)': '1',
  'vendor_id': 'GenuineIntel',
  'cpu_family': '6',
  'model': '158',
  'model_name': 'Intel(R) Core(TM) i7-8700K CPU @ 3.70GHz',
  'stepping': '10',
  'cpu_mhz': '3695.976',
  'bogomips': '7391.95',
  'hypervisor_vendor': 'KVM',
  'virtualization_type': 'full',
  'l1d_cache': '32K',
  'l1i_cache': '32K',
  'l2_cache': '256K',
  'l3_cache': '12288K',
  'numa_node0_cpu(s)': '0',
  'flags': 'fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 syscall nx rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq monitor ssse3 cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti fsgsbase avx2 invpcid 

In [14]:
stdin, stdout, stderr = client.exec_command('lshw -C display -json')

for line in stdout:
    print(line.strip('\n'))
    

                                        {
    "id" : "display",
    "class" : "display",
    "claimed" : true,
    "handle" : "PCI:0000:00:02.0",
    "description" : "VGA compatible controller",
    "product" : "VirtualBox Graphics Adapter",
    "vendor" : "InnoTek Systemberatung GmbH",
    "physid" : "2",
    "businfo" : "pci@0000:00:02.0",
    "version" : "00",
    "width" : 32,
    "clock" : 33000000,
    "configuration" : {
      "driver" : "vboxvideo",
      "latency" : "0"
    },
    "capabilities" : {
      "vga_controller" : true,
      "rom" : "extension ROM"
    }
  },
                                                      


In [15]:
import json 

def parse_lshw_stdout(stdout):
    json_stdout = ''
    for line in stdout:
        json_stdout += line.strip('\n')
    json_stdout = '[' + json_stdout.strip(",[ \t\r\n]+") + ']'
    return json.loads(json_stdout)


In [16]:
stdin, stdout, stderr = client.exec_command('lshw -C display -json')
parse_lshw_stdout(stdout)

[{'id': 'display',
  'class': 'display',
  'claimed': True,
  'handle': 'PCI:0000:00:02.0',
  'description': 'VGA compatible controller',
  'product': 'VirtualBox Graphics Adapter',
  'vendor': 'InnoTek Systemberatung GmbH',
  'physid': '2',
  'businfo': 'pci@0000:00:02.0',
  'version': '00',
  'width': 32,
  'clock': 33000000,
  'configuration': {'driver': 'vboxvideo', 'latency': '0'},
  'capabilities': {'vga_controller': True, 'rom': 'extension ROM'}}]

In [17]:
from collections import namedtuple


CMDHandler = namedtuple('CMDHandler', ['cmd', 'handle'])


class LinuxSystemInfoCrawler(SystemInfoCrawler):
    
    __operating_system_cmd_handlers = (
        CMDHandler('cat /etc/os-release', parse_cat_os_release_stdout),
        CMDHandler('hostnamectl', parse_hostnamectl_stdout)
    )
    
    __cpu_cmd_handlers = (
        CMDHandler('lscpu', parse_lscpu_stdout),
    )

    __gpu_cmd_handlers = (
        CMDHandler('lshw -C display -json', parse_lshw_stdout),
    )
    
    def __init__(self, ssh_client):
        self.__ssh_client = ssh_client

    def get_operating_system(self):
        return self.__gather_operating_system_info()
    
    def get_cpu(self):
        return self.__gather_cpu_info()

    
    def get_gpu(self):
        return self.__gather_gpu_info()
    
    def get_deamon_services(self):
        return []
    
    def get_software(self):
        return []
    
    def __gather_operating_system_info(self):
        os_info = {}
        for cmd_handler in self.__operating_system_cmd_handlers:
            os_info.update(self._read_remote_system_info(cmd_handler))
        return os_info
    
    def __gather_cpu_info(self):
        cpu_info = []
        for cmd_handler in self.__cpu_cmd_handlers:
            cpu_info += self._read_remote_system_info(cmd_handler)
        return cpu_info
    
    def __gather_gpu_info(self):
        gpu_info = []
        for cmd_handler in self.__gpu_cmd_handlers:
            gpu_info +=  self._read_remote_system_info(cmd_handler)
        return gpu_info
    
    def _read_remote_system_info(self, cmd_handler):
        self.__ssh_client.connect()
        
        _, stdout, _ = self.__ssh_client.exec_command(cmd_handler.cmd)
        system_info = cmd_handler.handle(stdout)
        
        self.__ssh_client.close()
        
        return system_info


In [18]:
crawler = LinuxSystemInfoCrawler(client)

In [19]:
crawler.get_operating_system()

{'name': 'Ubuntu',
 'version': '18.04.3 LTS (Bionic Beaver)',
 'id': 'ubuntu',
 'id_like': 'debian',
 'pretty_name': 'Ubuntu 18.04.3 LTS',
 'version_id': '18.04',
 'home_url': 'https://www.ubuntu.com/',
 'support_url': 'https://help.ubuntu.com/',
 'bug_report_url': 'https://bugs.launchpad.net/ubuntu/',
 'privacy_policy_url': 'https://www.ubuntu.com/legal/terms-and-policies/privacy-policy',
 'version_codename': 'bionic',
 'ubuntu_codename': 'bionic',
 'static_hostname': 'u1',
 'icon_name': 'computer-vm',
 'chassis': 'vm',
 'machine_id': 'ca6dfc4eb469493fa214dcef38a759f5',
 'boot_id': '1f0e0f02265545a7b98d01fcb71f4e60',
 'virtualization': 'oracle',
 'operating_system': 'Ubuntu 18.04.3 LTS',
 'kernel': 'Linux 5.0.0-23-generic',
 'architecture': 'x86-64'}

In [20]:
crawler.get_cpu()

[{'architecture': 'x86_64',
  'cpu_op-mode(s)': '32-bit, 64-bit',
  'byte_order': 'Little Endian',
  'cpu(s)': '1',
  'on-line_cpu(s)_list': '0',
  'thread(s)_per_core': '1',
  'core(s)_per_socket': '1',
  'socket(s)': '1',
  'numa_node(s)': '1',
  'vendor_id': 'GenuineIntel',
  'cpu_family': '6',
  'model': '158',
  'model_name': 'Intel(R) Core(TM) i7-8700K CPU @ 3.70GHz',
  'stepping': '10',
  'cpu_mhz': '3695.976',
  'bogomips': '7391.95',
  'hypervisor_vendor': 'KVM',
  'virtualization_type': 'full',
  'l1d_cache': '32K',
  'l1i_cache': '32K',
  'l2_cache': '256K',
  'l3_cache': '12288K',
  'numa_node0_cpu(s)': '0',
  'flags': 'fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 syscall nx rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq monitor ssse3 cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti fsgsbase avx2 invpcid 

In [21]:
crawler.get_gpu()

[{'id': 'display',
  'class': 'display',
  'claimed': True,
  'handle': 'PCI:0000:00:02.0',
  'description': 'VGA compatible controller',
  'product': 'VirtualBox Graphics Adapter',
  'vendor': 'InnoTek Systemberatung GmbH',
  'physid': '2',
  'businfo': 'pci@0000:00:02.0',
  'version': '00',
  'width': 32,
  'clock': 33000000,
  'configuration': {'driver': 'vboxvideo', 'latency': '0'},
  'capabilities': {'vga_controller': True, 'rom': 'extension ROM'}}]

In [40]:
def split_line_to_dict(line, by=';'):
    items = line.strip('\n').split(by)
    return {'name': items[0], 'version': items[-1]}

def parse_dpkg_query_stdout(stdout):
    return [split_line_to_dict(line) for line in stdout if is_line_splitable(line, r'[\;]')]


In [41]:
client.connect()
stdin, stdout, stderr = client.exec_command("dpkg-query -f '${binary:Package};${Version}\n' -W")

parse_dpkg_query_stdout(stdout)

[{'name': 'accountsservice', 'version': '0.6.45-1ubuntu1'},
 {'name': 'acl', 'version': '2.2.52-3build1'},
 {'name': 'acpi-support', 'version': '0.142'},
 {'name': 'acpid', 'version': '1:2.0.28-1ubuntu1'},
 {'name': 'adduser', 'version': '3.116ubuntu1'},
 {'name': 'adium-theme-ubuntu', 'version': '0.3.4-0ubuntu4'},
 {'name': 'adwaita-icon-theme', 'version': '3.28.0-1ubuntu1'},
 {'name': 'aisleriot', 'version': '1:3.22.5-1'},
 {'name': 'alsa-base', 'version': '1.0.25+dfsg-0ubuntu5'},
 {'name': 'alsa-utils', 'version': '1.1.3-1ubuntu1'},
 {'name': 'amd64-microcode', 'version': '3.20180524.1~ubuntu0.18.04.2'},
 {'name': 'anacron', 'version': '2.3-24'},
 {'name': 'apg', 'version': '2.2.3.dfsg.1-5'},
 {'name': 'app-install-data-partner', 'version': '16.04'},
 {'name': 'apparmor', 'version': '2.12-4ubuntu5.1'},
 {'name': 'apport', 'version': '2.20.9-0ubuntu7.7'},
 {'name': 'apport-gtk', 'version': '2.20.9-0ubuntu7.7'},
 {'name': 'apport-symptoms', 'version': '0.20'},
 {'name': 'appstream', '

In [42]:
def split_first_item(line):
    items = line.split(' ', 1)
    return {'name': items[0].strip()}

def parse_systemctl_list_units_stdout(stdout):
    return [split_first_item(line) for line in stdout]

In [43]:
stdin, stdout, stderr = client.exec_command('systemctl list-units --type service | grep ".service" | sed "s/\.service//g"')

parse_systemctl_list_units_stdout(stdout)

[{'name': 'accounts-daemon'},
 {'name': 'acpid'},
 {'name': 'apparmor'},
 {'name': 'apport'},
 {'name': 'avahi-daemon'},
 {'name': 'bolt'},
 {'name': 'colord'},
 {'name': 'console-setup'},
 {'name': 'cron'},
 {'name': 'cups-browsed'},
 {'name': 'cups'},
 {'name': 'dbus'},
 {'name': 'fwupd'},
 {'name': 'gdm'},
 {'name': 'grub-common'},
 {'name': 'kerneloops'},
 {'name': 'keyboard-setup'},
 {'name': 'kmod-static-nodes'},
 {'name': 'ModemManager'},
 {'name': 'networkd-dispatcher'},
 {'name': 'networking'},
 {'name': 'NetworkManager-wait-online'},
 {'name': 'NetworkManager'},
 {'name': 'packagekit'},
 {'name': 'polkit'},
 {'name': 'rsyslog'},
 {'name': 'rtkit-daemon'},
 {'name': 'setvtrgb'},
 {'name': 'snapd.seeded'},
 {'name': 'snapd'},
 {'name': 'speech-dispatcher'},
 {'name': 'ssh'},
 {'name': 'systemd-journal-flush'},
 {'name': 'systemd-journald'},
 {'name': 'systemd-logind'},
 {'name': 'systemd-modules-load'},
 {'name': 'systemd-random-seed'},
 {'name': 'systemd-remount-fs'},
 {'name'

In [44]:
class DebianInfoCrawler(LinuxSystemInfoCrawler):
    __software_cmd_handlers = (
        CMDHandler("dpkg-query -f '${binary:Package};${Version}\n' -W", 
                   parse_dpkg_query_stdout),
    )
    
    __deamon_services_cmd_handlers = (
        CMDHandler('systemctl list-units --type service | grep ".service" | sed "s/\.service//g"', 
                   parse_systemctl_list_units_stdout),
    )
    
    def get_software(self):
        return self.__gather_software_info()
    
    def get_deamon_services(self):
        return self.__gather_deamon_services_info()
    
    def __gather_software_info(self):
        software_info = []
        for cmd_handler in self.__software_cmd_handlers:
            software_info +=  self._read_remote_system_info(cmd_handler)
        return software_info

    def __gather_deamon_services_info(self):
        deamon_services_info = []
        for cmd_handler in self.__deamon_services_cmd_handlers:
            deamon_services_info +=  self._read_remote_system_info(cmd_handler)
        return deamon_services_info


In [45]:
crawler = DebianInfoCrawler(client)

In [48]:
crawler.get_operating_system()

{'name': 'Ubuntu',
 'version': '18.04.3 LTS (Bionic Beaver)',
 'id': 'ubuntu',
 'id_like': 'debian',
 'pretty_name': 'Ubuntu 18.04.3 LTS',
 'version_id': '18.04',
 'home_url': 'https://www.ubuntu.com/',
 'support_url': 'https://help.ubuntu.com/',
 'bug_report_url': 'https://bugs.launchpad.net/ubuntu/',
 'privacy_policy_url': 'https://www.ubuntu.com/legal/terms-and-policies/privacy-policy',
 'version_codename': 'bionic',
 'ubuntu_codename': 'bionic',
 'static_hostname': 'u1',
 'icon_name': 'computer-vm',
 'chassis': 'vm',
 'machine_id': 'ca6dfc4eb469493fa214dcef38a759f5',
 'boot_id': '1f0e0f02265545a7b98d01fcb71f4e60',
 'virtualization': 'oracle',
 'operating_system': 'Ubuntu 18.04.3 LTS',
 'kernel': 'Linux 5.0.0-23-generic',
 'architecture': 'x86-64'}

In [49]:
crawler.get_cpu()

[{'architecture': 'x86_64',
  'cpu_op-mode(s)': '32-bit, 64-bit',
  'byte_order': 'Little Endian',
  'cpu(s)': '1',
  'on-line_cpu(s)_list': '0',
  'thread(s)_per_core': '1',
  'core(s)_per_socket': '1',
  'socket(s)': '1',
  'numa_node(s)': '1',
  'vendor_id': 'GenuineIntel',
  'cpu_family': '6',
  'model': '158',
  'model_name': 'Intel(R) Core(TM) i7-8700K CPU @ 3.70GHz',
  'stepping': '10',
  'cpu_mhz': '3695.976',
  'bogomips': '7391.95',
  'hypervisor_vendor': 'KVM',
  'virtualization_type': 'full',
  'l1d_cache': '32K',
  'l1i_cache': '32K',
  'l2_cache': '256K',
  'l3_cache': '12288K',
  'numa_node0_cpu(s)': '0',
  'flags': 'fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 syscall nx rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq monitor ssse3 cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti fsgsbase avx2 invpcid 

In [50]:
crawler.get_gpu()

[{'id': 'display',
  'class': 'display',
  'claimed': True,
  'handle': 'PCI:0000:00:02.0',
  'description': 'VGA compatible controller',
  'product': 'VirtualBox Graphics Adapter',
  'vendor': 'InnoTek Systemberatung GmbH',
  'physid': '2',
  'businfo': 'pci@0000:00:02.0',
  'version': '00',
  'width': 32,
  'clock': 33000000,
  'configuration': {'driver': 'vboxvideo', 'latency': '0'},
  'capabilities': {'vga_controller': True, 'rom': 'extension ROM'}}]

In [46]:
crawler.get_software()

[{'name': 'accountsservice', 'version': '0.6.45-1ubuntu1'},
 {'name': 'acl', 'version': '2.2.52-3build1'},
 {'name': 'acpi-support', 'version': '0.142'},
 {'name': 'acpid', 'version': '1:2.0.28-1ubuntu1'},
 {'name': 'adduser', 'version': '3.116ubuntu1'},
 {'name': 'adium-theme-ubuntu', 'version': '0.3.4-0ubuntu4'},
 {'name': 'adwaita-icon-theme', 'version': '3.28.0-1ubuntu1'},
 {'name': 'aisleriot', 'version': '1:3.22.5-1'},
 {'name': 'alsa-base', 'version': '1.0.25+dfsg-0ubuntu5'},
 {'name': 'alsa-utils', 'version': '1.1.3-1ubuntu1'},
 {'name': 'amd64-microcode', 'version': '3.20180524.1~ubuntu0.18.04.2'},
 {'name': 'anacron', 'version': '2.3-24'},
 {'name': 'apg', 'version': '2.2.3.dfsg.1-5'},
 {'name': 'app-install-data-partner', 'version': '16.04'},
 {'name': 'apparmor', 'version': '2.12-4ubuntu5.1'},
 {'name': 'apport', 'version': '2.20.9-0ubuntu7.7'},
 {'name': 'apport-gtk', 'version': '2.20.9-0ubuntu7.7'},
 {'name': 'apport-symptoms', 'version': '0.20'},
 {'name': 'appstream', '

In [47]:
crawler.get_deamon_services()

[{'name': 'accounts-daemon'},
 {'name': 'acpid'},
 {'name': 'apparmor'},
 {'name': 'apport'},
 {'name': 'avahi-daemon'},
 {'name': 'bolt'},
 {'name': 'colord'},
 {'name': 'console-setup'},
 {'name': 'cron'},
 {'name': 'cups-browsed'},
 {'name': 'cups'},
 {'name': 'dbus'},
 {'name': 'fwupd'},
 {'name': 'gdm'},
 {'name': 'grub-common'},
 {'name': 'kerneloops'},
 {'name': 'keyboard-setup'},
 {'name': 'kmod-static-nodes'},
 {'name': 'ModemManager'},
 {'name': 'networkd-dispatcher'},
 {'name': 'networking'},
 {'name': 'NetworkManager-wait-online'},
 {'name': 'NetworkManager'},
 {'name': 'packagekit'},
 {'name': 'polkit'},
 {'name': 'rsyslog'},
 {'name': 'rtkit-daemon'},
 {'name': 'setvtrgb'},
 {'name': 'snapd.seeded'},
 {'name': 'snapd'},
 {'name': 'speech-dispatcher'},
 {'name': 'ssh'},
 {'name': 'systemd-journal-flush'},
 {'name': 'systemd-journald'},
 {'name': 'systemd-logind'},
 {'name': 'systemd-modules-load'},
 {'name': 'systemd-random-seed'},
 {'name': 'systemd-remount-fs'},
 {'name'

In [29]:
class CentOSInfoCrawler(LinuxSystemInfoCrawler):
    def get_updates(self):
        return
    
    def get_software(self):
        return

    
class FedoraInfoCrawler(LinuxSystemInfoCrawler):
    def get_updates(self):
        return
    
    def get_software(self):
        return

    
class RedHatInfoCrawler(LinuxSystemInfoCrawler):
    def get_updates(self):software
        return
    
    def get_software(self):
        return

    
class FreeBSDInfoCrawler(LinuxSystemInfoCrawler):
    def get_updates(self):
        return
    
    def get_software(self):
        return

class OpenBSDInfoCrawler(LinuxSystemInfoCrawler):
    def get_updates(self):
        return
    
    def get_software(self):
        return